In [1]:
# | default_exp jira.auth

In [2]:
# | exporti
import urllib.parse as parse
from dataclasses import dataclass, field
import base64
import httpx

import gdoc_sync.client as gd

In [3]:
# | hide
from nbdev.showdoc import *
import os
from dotenv import load_dotenv

load_dotenv("../.env")

True

In [4]:
# | exporti
@dataclass
class _JiraAuth_Required:
    instance: str  # 'your-domain.atlassian.net'


@dataclass
class _JiraAuth_Optional:
    base_url: str = None

In [5]:
# | export


@dataclass
class JiraAuth(_JiraAuth_Optional, _JiraAuth_Required, gd.Auth):
    pass

In [6]:
# | exporti
@dataclass
class _JiraAuthPat_Required:
    email: str
    pat: str = field(repr=False)

In [7]:
# | export
@dataclass
class JiraAuthPat(_JiraAuth_Optional, _JiraAuthPat_Required, _JiraAuth_Required):
    def __post_init__(self):
        if not self.instance.startswith("https://"):
            instance = f"https://{self.instance}"

        self.instance = parse.urlparse(instance).netloc
        self.base_url = instance

    def generate_auth_header(self):
        auth_str = base64.b64encode(f"{self.email}:{self.pat}".encode("ascii")).decode(
            "ascii"
        )
        return {"Authorization": f"Basic {auth_str}"}

    async def who_am_i(self, client: httpx.AsyncClient = None, debug_api: bool = False):
        url = f"{self.base_url}/rest/api/2/myself"

        response = await gd.get_data(
            method="GET",
            url=url,
            auth=self,
            debug_api=debug_api,
            client=client,
        )
        return response

In [8]:
import os

jira_pat = os.environ["JIRA_PAT"]
jira_instance = os.environ["JIRA_INSTANCE"]

jira_auth = JiraAuthPat(
    pat=jira_pat, email="jae@onyxreporting.com", instance=jira_instance
)

res = await jira_auth.who_am_i(debug_api=False)

res.response

jira_auth

JiraAuthPat(instance='onyxreporting.atlassian.net', email='jae@onyxreporting.com', base_url='https://onyxreporting.atlassian.net')

In [9]:
# | hide
import nbdev

nbdev.nbdev_export()